# Cleaning data workflow
1. Save originals as `<state-abbrev>`-alf-`<type>`-raw.csv
2. Hand clean to `<state-abbrev>`-alf-`<type>`-hc.csv
    - remove not used information
    - change column names
4. Import into pandas, clean some more
    - bespoke changes
        - add type information
        - add date accessed
        - parse weird data
        - combine into one dataset per state
    - use cleaner function
        - remove duplicates
        - remove wrong columns

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata

In [2]:
columns = ['Address',
 'Capacity',
 'City',
 'County',
 'Date Accessed',
 'Email Address',
 'Facility ID',
 'Facility Name',
 'Latitude',
 'License Number',
 'Licensee',
 'Longitude',
 'Ownership Type',
 'Phone Number',
 'State',
 'State Facility Type 1 Literal',
 'State Facility Type 2 Literal',
 'Zip Code']

cols_to_title = ['City', 'County', 'Facility Name']
cols_to_lower = ['Email Address']
cols_to_upper = ['Address', 'State Facility Type 1 Literal', 'State Facility Type 2 Literal', 'Ownership Type', 'Licensee']

In [3]:
# cleaner function
# adds state abbreviation
# fixes capitalization
# removes some duplicates and shows you how many remaining
def cleaner(df, state):
    # finding wrong columns
    print('Extraneous columns are:\n{}'.format(list(set(df.columns) - set(columns))))

    df['State'] = state

    # fixing capitalization
    for col in df.columns:
        if   col in cols_to_title: df[col] = df[col].str.title()
        elif col in cols_to_lower: df[col] = df[col].str.lower()
        elif col in cols_to_upper: df[col] = df[col].str.upper()


    if 'License Number' in df.columns:
        df.drop_duplicates(['Facility Name', 'License Number'], inplace=True)
        print('{} ALFs, {} unique License Numbers, {} NA License Numbers'.format(df.shape[0], df['License Number'].unique().shape[0], df['License Number'].isna().sum()))
    elif 'Facility ID' in df.columns:
        df.drop_duplicates(['Facility Name', 'Facility ID'], inplace=True)
        print('{} ALFs, {} unique Facility IDs, {} NA Facility IDs'.format(df.shape[0], df['Facility ID'].unique().shape[0], df['Facility ID'].isna().sum()))
    else:
        df.drop_duplicates(['Facility Name', 'Address'], inplace=True)
        print('No License Number or Facility ID')

## Arkansas

In [12]:
! ls ../alf-datasets/ar

ar-alf-hc.csv  ar-alf-raw.csv


In [13]:
df_ar = pd.read_csv('../alf-datasets/ar/ar-alf-hc.csv')

In [14]:
df_ar['Date Accessed'] = '8/27/21'

In [15]:
cleaner(df_ar, 'AR')

Extraneous columns are:
[]
104 ALFs, 104 unique Facility IDs, 0 NA Facility IDs


In [16]:
df_ar.to_csv('../alf-datasets/ar/ar-alf-final.csv', index=False)

## Virgina

In [7]:
! ls ../alf-datasets/va

va-alf-hc.csv  va-alf-raw.csv


In [8]:
df_va = pd.read_csv('../alf-datasets/va/va-alf-hc.csv')

In [9]:
df_va['Date Accessed'] = '8/31/21'
df_va['State Facility Type 1 Literal'] = 'Assisted Living Facility'

In [10]:
cleaner(df_va, 'VA')

Extraneous columns are:
[]
No License Number or Facility ID


In [11]:
df_va.to_csv('../alf-datasets/va/va-alf-final.csv', index=False)

## North Dakota

In [17]:
! ls ../alf-datasets/nd

nd-alf-al-hc.csv    nd-alf-al.pdf       nd-alf-bc.pdf
nd-alf-al-raw.csv   nd-alf-bc-hc.xlsx   ~$nd-alf-bc-hc.xlsx


In [26]:
df_nd_al = pd.read_csv('../alf-datasets/nd/nd-alf-al-hc.csv')
df_nd_al['State Facility Type 1 Literal'] = 'assisted living facility'
df_nd_al['Date Accessed'] = '8/17/21'

df_nd_bc = pd.read_csv('../alf-datasets/nd/nd-alf-bc-hc.csv')
df_nd_bc['State Facility Type 1 Literal'] = 'basic care facility'
df_nd_bc['Date Accessed'] = '8/17/21'

In [27]:
df_nd = pd.concat([df_nd_al, df_nd_bc], ignore_index=True)

In [30]:
cleaner(df_nd, 'ND')

Extraneous columns are:
[]
139 ALFs, 65 unique Facility IDs, 74 NA Facility IDs


In [32]:
df_nd.to_csv('../alf-datasets/nd/nd-alf-final.csv', index=False)

## Mississippi

In [976]:
! ls alf-datasets/ms

ms-alf-hc.csv  ms-alf-raw.csv ms-alf.pdf


In [977]:
df_ms = pd.read_csv('alf-datasets/ms/ms-alf-hc.csv')

In [978]:
df_ms['State Facility Type 1 Literal'] = 'personal care home'
df_ms['State Facility Type 2 Literal'] = ['assisted living' if type(x) is str else 'residential living' for x in df_ms['Assisted Living']]
df_ms['City'] = df_ms['Location'].str.split(',', expand=True)[0]
df_ms['Zip Code'] = df_ms['Location'].str[-5:]
df_ms['Date Accessed'] = '8/11/21'

In [979]:
df_ms.drop(columns=['Assisted Living', 'Location'], inplace=True)

In [980]:
cleaner(df_ms, 'MS')

Extraneous columns are:
[]
No License Number or Facility ID


In [982]:
df_ms.to_csv('alf-datasets/ms/ms-alf-final.csv', index=False)

## Washington DC

In [633]:
! ls alf-datasets/dc

dc-alf-manual.csv


In [634]:
df_dc = pd.read_csv('alf-datasets/dc/dc-alf-manual.csv')
cleaner(df_dc, 'DC')

Extraneous columns are:
[]
No License Number of Facility ID


In [636]:
df_dc.to_csv('alf-datasets/dc/dc-alf-final.csv', index=False)

## Wyoming

In [489]:
# NEED TO FIGURE THIS ONE OUT
df_wy = pd.read_csv('alf-datasets/wy/wy-alf-hc.csv')

In [490]:
a = df_wy['Name\nPhysical Address'].str.split('\n', expand=True)
# a.head(20)

In [491]:
def wyoming_helper_function(row):
    if row[4] is None:
        row[4] = row[3]
        row[3] = row[2]
        row[2] = row[1]
        row[1] = row[0]
        row[0] = ''
    return row

In [492]:
a = a.apply(test_function, axis=1)

In [493]:
df_wy['Facility Name'] = [x[4:] if "dba/" in x else x for x in a[1]]
df_wy['Licensee'] = a[0]
df_wy['Address'] = a[2]
df_wy['City'] = a[3].str.split(' ', expand=True)[0]
df_wy['Zip Code'] = a[3].str.split(' ', expand=True)[2]
df_wy['County'] = df_wy['County\nFiscal Year End'].str.split('\n', expand=True)[0]
df_wy['Capacity'] = df_wy['# of Licensed Beds'].str.split('\n', expand=True)[0].str.split(' ', expand=True)[0]
df_wy['Capacity'].iloc[16] = 34 # manually fixing one entry
df_wy['Phone Number'] = df_wy['Phone\nFax'].str.split('\n', expand=True)[0]
df_wy['State Facility Type 1 Literal'] = 'Assisted living facility'

In [494]:
df_wy.drop(columns=["Name\nPhysical Address", "Phone\nFax", "County\nFiscal Year End", "# of Licensed Beds"], inplace=True)

In [495]:
cleaner(df_wy, 'WY')

Extraneous columns are:
[]
33 ALFs, 33 unique License Numbers, 0 NA License Numbers


In [497]:
df_wy.to_csv('alf-datasets/wy/wy-alf-final.csv', index=False)

## Wisconsin

In [373]:
! ls alf-datasets/wi/

wi-alf-afh-hc.csv   wi-alf-cbrf-hc.csv  wi-alf-final.csv    wi-alf-rcac-raw.csv
wi-alf-afh-raw.csv  wi-alf-cbrf-raw.csv wi-alf-rcac-hc.csv


In [374]:
df_wi_cbrf = pd.read_csv('alf-datasets/wi/wi-alf-cbrf-hc.csv')
df_wi_rcac = pd.read_csv('alf-datasets/wi/wi-alf-rcac-hc.csv')
df_wi_afh = pd.read_csv('alf-datasets/wi/wi-alf-afh-hc.csv')

In [375]:
df_wi_cbrf['State Facility Type 1 Literal'] = 'Community-Based Residential Facility'
df_wi_rcac['State Facility Type 1 Literal'] = 'Residential Care Apartment Complex'
df_wi_afh['State Facility Type 1 Literal'] = 'adult family home'
df_wi = pd.concat([df_wi_cbrf, df_wi_rcac, df_wi_afh], ignore_index=True)
df_wi['Date Accessed'] = '7/6/21'

In [376]:
df_wi.head()

,Facility ID,Facility Name,Address,City,State,Zip Code,Phone Number,County,Licensee,State Facility Type 2 Literal,State Facility Type 1 Literal,Capacity,Date Accessed
0,15835,LIBERTY VILLAGE OF ADAMS,550 W LIBERTY STREET,ADAMS,WI,53910,(608) 339-9444,ADAMS,COPPERLEAF MANAGEMENT GROUP INC,CLASS CNA (NONAMBULATORY),Community-Based Residential Facility,NaN,7/6/21
1,15012,BIRCH HAVEN SENIOR LIVING BEARS HOLLOW,1019 15TH AVE W,ASHLAND,WI,54806-,(715) 682-2534,ASHLAND,KELM ENTERPRISES INC,CLASS CNA (NONAMBULATORY),Community-Based Residential Facility,NaN,7/6/21
2,15013,BIRCH HAVEN SENIOR LIVING EAGLES RIDGE,224 22ND AVE W,ASHLAND,WI,54806-,(715) 682-1069,ASHLAND,KELM ENTERPRISES INC,CLASS CNA (NONAMBULATORY),Community-Based Residential Facility,NaN,7/6/21
3,15014,BIRCH HAVEN SENIOR LIVING FALCONS CREST,218 22ND AVE W,ASHLAND,WI,54806-,(715) 682-2775,ASHLAND,KELM ENTERPRISES INC,CLASS CNA (NONAMBULATORY),Community-Based Residential Facility,NaN,7/6/21
4,15011,BIRCH HAVEN SENIOR LIVING TIMBERS EDGE,1500 10TH ST W,ASHLAND,WI,54806-,(715) 682-2506,ASHLAND,KELM ENTERPRISES INC,CLASS CNA (NONAMBULATORY),Community-Based Residential Facility,NaN,7/6/21


In [377]:
cleaner(df_wi, 'WI')

Extraneous columns are:
[]
4061 ALFs, 4061 unique Facility IDs, 0 NA Facility IDs


In [378]:
df_wi.to_csv('alf-datasets/wi/wi-alf-final.csv', index=False)

## West Virginia

In [873]:
! ls alf-datasets/wv

wv-alf-al-raw.csv wv-alf-final.csv  wv-alf-hc.csv     wv-alf-rc-raw.csv


In [874]:
df_wv = pd.read_csv('alf-datasets/wv/wv-alf-hc.csv')
df_wv['Date Accessed'] = '7/6/21'
cleaner(df_wv, 'WV')

Extraneous columns are:
['Medicaid', 'Medicare']
95 ALFs, 83 unique License Numbers, 12 NA License Numbers


In [875]:
df_wv = df_wv.drop(columns=['Medicaid', 'Medicare'])

In [876]:
df_wv.to_csv('alf-datasets/wv/wv-alf-final.csv', index=False)

## Washington

In [877]:
! ls alf-datasets/wa

wa-alf-afh-(not-currently-using).csv wa-alf-al-raw.csv
wa-alf-al-hc.csv                     wa-alf-final.csv


In [878]:
df_wa = pd.read_csv('alf-datasets/wa/wa-alf-al-hc.csv')
df_wa['Date Accessed'] = '7/6/21'
cleaner(df_wa, 'WA')

Extraneous columns are:
['Medicaid']
541 ALFs, 541 unique License Numbers, 0 NA License Numbers


In [879]:
df_wa = df_wa.drop(columns=['Medicaid'])

In [880]:
df_wa.to_csv('alf-datasets/wa/wa-alf-final.csv', index=False)

## Utah

In [370]:
! ls alf-datasets/ut

ut-alf-final.csv ut-alf-hc.csv    ut-alf-raw.csv


In [371]:
df_ut = pd.read_csv('alf-datasets/ut/ut-alf-hc.csv')
df_ut['Date Accessed'] = '7/6/21'
df_ut = df_ut.rename(columns={'Zip':'Zip Code', 'State Facility Type 1 Literal':'State Facility Type 2 Literal'})
df_ut['State Facility Type 1 Literal'] = 'assisted living facility'

cleaner(df_ut, 'UT')

Extraneous columns are:
[]
240 ALFs, 240 unique License Numbers, 0 NA License Numbers


In [372]:
df_ut.to_csv('alf-datasets/ut/ut-alf-final.csv', index=False)

## Texas

In [885]:
! ls alf-datasets/tx

tx-alf-final.csv tx-alf-hc.csv    tx-alf-raw.csv


In [886]:
df_tx = pd.read_csv('alf-datasets/tx/tx-alf-hc.csv')
df_tx['Date Accessed'] = '7/6/21'
cleaner(df_tx, 'TX')

Extraneous columns are:
['Geo Location']
2024 ALFs, 2024 unique License Numbers, 0 NA License Numbers


In [887]:
df_tx[['Latitude', 'Longitude']] = df_tx['Geo Location'].str.split(',', expand=True)

In [888]:
df_tx = df_tx.drop(columns=['Geo Location'])

In [889]:
df_tx.to_csv('alf-datasets/tx/tx-alf-final.csv', index=False)

## Tennessee

In [890]:
! ls alf-datasets/tn

tn-alf-ac-hc.csv  tn-alf-ac.pdf     tn-alf-rh-hc.csv  tn-alf-rh.pdf
tn-alf-ac-raw.csv tn-alf-final.csv  tn-alf-rh-raw.csv


In [891]:
df_tn_ac = pd.read_csv('alf-datasets/tn/tn-alf-ac-hc.csv')
df_tn_rh = pd.read_csv('alf-datasets/tn/tn-alf-rh-hc.csv')

df_tn_rh['State Facility Type 1 Literal'] = 'Residential Home for the Aged'
df_tn_ac['State Facility Type 1 Literal'] = 'Assisted Care Living Facility'

df_tn = pd.concat([df_tn_rh, df_tn_ac], ignore_index=True)
df_tn['Date Accessed'] = '6/30/21'


In [892]:
df_tn['Facility Name'] = df_tn['Facility Name'].str.replace('\n', ' ')

split = df_tn['Address'].str.split('\n', expand=True)
split2 = split[1].str.split(', ', expand=True)
split3 = split2[1].str.split(' ', expand=True)

df_tn['Address'] = split[0]
df_tn['City'] = split2[0]
df_tn['Zip Code'] = split3[1]
df_tn['State'] = 'TN'

In [893]:
cleaner(df_tn, 'TN')

Extraneous columns are:
[]
373 ALFs, 351 unique License Numbers, 0 NA License Numbers


In [894]:
# df_tn.sort_values(by=['License Number']).tail(30)
# duplicate License Numbers are irrelevant

In [895]:
df_tn.to_csv('alf-datasets/tn/tn-alf-final.csv', index=False)

## South Dakota

In [896]:
! ls alf-datasets/sd

sd-alf-final.csv sd-alf-hc.csv    sd-alf-raw.csv


In [897]:
df_sd = pd.read_csv('alf-datasets/sd/sd-alf-hc.csv')
df_sd['Date Accessed'] = '6/30/21'
df_sd['State Facility Type 1 Literal'] = 'Assisted Living Center'
cleaner(df_sd, 'SD')

Extraneous columns are:
['City.1']
163 ALFs, 163 unique License Numbers, 0 NA License Numbers


In [898]:
df_sd = df_sd.drop(columns='City.1')

In [899]:
df_sd.to_csv('alf-datasets/sd/sd-alf-final.csv', index=False)

## South Carolina

In [900]:
! ls alf-datasets/sc

sc-alf-final.csv sc-alf-hc.csv    sc-alf-raw.csv


In [901]:
df_sc = pd.read_csv('alf-datasets/sc/sc-alf-hc.csv')
df_sc['Date Accessed'] = '6/30/21'
df_sc['State Facility Type 1 Literal'] = 'Community Residential Care Facilities'

In [902]:
df_sc['License Number'] = df_sc['License / Expiration'].str.split(' / ', expand=True)[0]
df_sc['Ownership Type'] = df_sc['Ownership Type'].str.split(' / ', expand=True)[1]
df_sc = df_sc.drop(columns=['License / Expiration'])

In [903]:
cleaner(df_sc, 'SC')

Extraneous columns are:
[]
484 ALFs, 484 unique License Numbers, 0 NA License Numbers


In [904]:
df_sc.to_csv('alf-datasets/sc/sc-alf-final.csv', index=False)

## Rhode Island

In [905]:
! ls alf-datasets/ri

ri-alf-final.csv ri-alf-hc.csv    ri-alf-raw.csv


In [906]:
df_ri = pd.read_csv('alf-datasets/ri/ri-alf-hc.csv')
df_ri['Date Accessed'] = '6/30/21'
cleaner(df_ri, 'RI')

Extraneous columns are:
['Zip']
65 ALFs, 65 unique License Numbers, 0 NA License Numbers


In [907]:
df_ri = df_ri.rename(columns={'Zip':'Zip Code'})

In [908]:
df_ri.to_csv('alf-datasets/ri/ri-alf-final.csv', index=False)

## Pennsylvania

In [355]:
! ls alf-datasets/pa

pa-alf-alr-hc.csv  pa-alf-final.csv   pa-alf-pch-hc.csv
pa-alf-alr-raw.csv pa-alf-notes.pdf   pa-alf-pch-raw.csv


In [356]:
df_pa_pch = pd.read_csv('alf-datasets/pa/pa-alf-pch-hc.csv')
df_pa_alr = pd.read_csv('alf-datasets/pa/pa-alf-alr-hc.csv')

df_pa_pch['State Facility Type 1 Literal'] = 'Personal Care Home'
df_pa_alr['State Facility Type 1 Literal'] = 'Assisted Living Residence'

df_pa = pd.concat([df_pa_alr, df_pa_pch], ignore_index=True)
df_pa['Date Accessed'] = '6/30/21'

In [357]:
df_pa = df_pa.rename(columns={'Zip':'Zip Code'})

In [358]:
cleaner(df_pa, 'PA')

Extraneous columns are:
[]
1159 ALFs, 1159 unique License Numbers, 0 NA License Numbers


In [359]:
df_pa.to_csv('alf-datasets/pa/pa-alf-final.csv', index=False)

## Oregon

In [913]:
! ls alf-datasets/or

or-alf-final.csv or-alf-hc.csv    or-alf-raw.csv


In [914]:
df_or = pd.read_csv('alf-datasets/or/or-alf-hc.csv')
df_or['Date Accessed'] = '7/6/21'
df_or['State Facility Type 1 Literal'] = ['Assisted Living Facility' if x == 'ALF' else 'Residential Care Facility' for x in df_or['State Facility Type 1 Literal']]
cleaner(df_or, 'OR')

Extraneous columns are:
['Medicaid']
562 ALFs, 562 unique Facility IDs, 0 NA Facility IDs


In [915]:
df_or = df_or.drop(columns='Medicaid')

In [916]:
df_or.to_csv('alf-datasets/or/or-alf-final.csv', index=False)

## Oklahoma

In [352]:
! ls alf-datasets/ok

ok-alf-al-hc.csv  ok-alf-al.pdf     ok-alf-rc-hc.csv  ok-alf-rc.pdf
ok-alf-al-raw.csv ok-alf-final.csv  ok-alf-rc-raw.csv


In [353]:
df_ok_al = pd.read_csv('alf-datasets/ok/ok-alf-al-hc.csv')
# df_ok_rc = pd.read_csv('alf-datasets/ok/ok-alf-rc-hc.csv')
# df_ok = pd.concat([df_ok_al, df_ok_rc], ignore_index=True)
df_ok = df_ok_al

df_ok['Date Accessed'] = '7/2/21'
cleaner(df_ok, 'OK')

Extraneous columns are:
[]
189 ALFs, 189 unique Facility IDs, 0 NA Facility IDs


In [354]:
df_ok.to_csv('alf-datasets/ok/ok-alf-final.csv', index=False)

## Ohio

In [938]:
! ls alf-datasets/oh

oh-alf-final.csv oh-alf-hc.csv    oh-alf-raw.csv


In [939]:
df_oh = pd.read_csv('alf-datasets/oh/oh-alf-hc.csv')
df_oh['Date Accessed'] = '6/29/21'

In [940]:
df_oh.shape

(1155, 14)

In [941]:
cleaner(df_oh, 'OH')

Extraneous columns are:
[]
793 ALFs, 793 unique License Numbers, 0 NA License Numbers


In [942]:
df_oh.to_csv('alf-datasets/oh/oh-alf-final.csv', index=False)

## North Carolina

In [831]:
! ls alf-datasets/nc

nc-alf-ach-hc.csv  nc-alf-ach-raw.csv nc-alf-final.csv   nc-alf-mahs.html


In [832]:
df_nc_ach = pd.read_csv('alf-datasets/nc/nc-alf-ach-hc.csv')
cleaner(df_nc_ach, 'NC')

Extraneous columns are:
[]
594 ALFs, 594 unique License Numbers, 0 NA License Numbers


In [833]:
# there's more NC at the bottom to be dealt with
# df_nc.to_csv('alf-datasets/nc/nc-alf-final.csv', index=False)

## New York

In [631]:
! ls alf-datasets/ny

ny-alf-final.csv ny-alf-hc.csv    ny-alf-raw.csv


In [632]:
df_ny = pd.read_csv("alf-datasets/ny/ny-alf-hc.csv")
df_ny["Date Accessed"] = "6/29/21"

In [951]:
df_ny['Type'].unique()

array(['AH/ALP', 'AH', 'EHP', 'EHP/ALP'], dtype=object)

In [952]:
def type_simplifier_ny(x):
    if x == 'AH/ALP': return 'Adult Home and Assisted Living Program'
    elif x == 'AH': return 'Adult Home'
    elif x == 'EHP': return 'Enriched Housing Program'
    elif x == 'EHP/ALP': return 'Enriched Housing and Assisted Living Program'
    else: return 'THIS IS A MISTAKE'

In [953]:
df_ny['State Facility Type 1 Literal'] = 'Adult Care Facility'
df_ny['State Facility Type 2 Literal'] = [type_simplifier_ny(x) for x in df_ny['Type']]

In [954]:
df_ny['Ownership Type'] = ['Non Profit' if x == 'NFP' else 'For Profit' for x in df_ny['Classification']]

In [955]:
df_ny = df_ny.drop(columns=['Type', 'Classification'])

In [956]:
cleaner(df_ny, 'NY')

Extraneous columns are:
[]
553 ALFs, 553 unique License Numbers, 0 NA License Numbers


In [958]:
df_ny.to_csv('alf-datasets/ny/ny-alf-final.csv', index=False)

## New Jersey

In [959]:
! ls alf-datasets/nj

nj-alf-final.csv nj-alf-hc.csv    nj-alf-raw.csv


In [960]:
df_nj = pd.read_csv('alf-datasets/nj/nj-alf-hc.csv')
df_nj['Date Accessed'] = '6/29/21'
df_nj['State Facility Type 1 Literal'] = 'Assisted Living Service'
cleaner(df_nj, 'NJ')

Extraneous columns are:
[]
269 ALFs, 269 unique License Numbers, 0 NA License Numbers


In [962]:
df_nj.to_csv('alf-datasets/nj/nj-alf-final.csv', index=False)

## Nevada

In [963]:
! ls alf-datasets/nv

nv-alf-final.csv nv-alf-hc.csv    nv-alf-raw.csv


In [964]:
df_nv = pd.read_csv('alf-datasets/nv/nv-alf-hc.csv')
df_nv['Date Accessed'] = '6/29/21'
cleaner(df_nv, 'NV')

Extraneous columns are:
[]
387 ALFs, 387 unique License Numbers, 0 NA License Numbers


In [966]:
df_nv.to_csv('alf-datasets/nv/nv-alf-final.csv', index=False)

## Nebraska

In [967]:
! ls alf-datasets/ne

ne-alf-final.csv ne-alf-hc.csv    ne-alf-raw.csv


In [968]:
df_ne = pd.read_csv('alf-datasets/ne/ne-alf-hc.csv')
df_ne['Date Accessed'] = '6/29/21'
cleaner(df_ne, 'NE')

Extraneous columns are:
[]
No License Number of Facility ID


In [970]:
df_ne.to_csv('alf-datasets/ne/ne-alf-final.csv', index=False)

## Missouri

In [983]:
! ls alf-datasets/mo

mo-alf-final.csv mo-alf-hc.csv    mo-alf-raw.csv


In [984]:
df_mo = pd.read_csv('alf-datasets/mo/mo-alf-hc.csv')
df_mo['Date Accessed'] = '6/29/21'

In [985]:
df_mo['Level of Care'].unique()

array(['RCF', 'SNF', 'ALF', 'ICF', 'RCF*', 'ALF**'], dtype=object)

In [986]:
df_mo = df_mo[df_mo['Level of Care'].isin(['RCF', 'ALF', 'RCF*', 'ALF**'])]

In [987]:
def type_simplifier_mo(x):
    if x == 'RCF': return 'Residential Care Facility'
    elif x == 'ALF': return 'Assisted Living Facility'
    elif x == 'RCF*': return 'Residential Care Facility*'
    elif x == 'ALF**': return 'Assisted Living Facility**'
    else: return 'THIS IS A MISTAKE'

In [988]:
df_mo['State Facility Type 1 Literal'] = [type_simplifier_mo(x) for x in df_mo['Level of Care']]
df_mo = df_mo.rename(columns={'Level of Care Code': 'State Facility Type 2 Literal'})
df_mo = df_mo.drop(columns='Level of Care')

In [989]:
cleaner(df_mo, 'MO')

Extraneous columns are:
[]
632 ALFs, 631 unique Facility IDs, 0 NA Facility IDs


In [990]:
df_mo.to_csv('alf-datasets/mo/mo-alf-final.csv', index=False)

## Minnesota

In [346]:
! ls alf-datasets/mn

mn-alf-final.csv   mn-alf-hws-raw.csv mn-alf-sl-raw.csv
mn-alf-hws-hc.csv  mn-alf-sl-hc.csv


In [347]:
df_mn_sl = pd.read_csv('alf-datasets/mn/mn-alf-sl-hc.csv')
df_mn_hws = pd.read_csv('alf-datasets/mn/mn-alf-hws-hc.csv')
df_mn = pd.concat([df_mn_sl, df_mn_hws], ignore_index=True)
df_mn['Date Accessed'] = '7/12/21'
cleaner(df_mn, 'MN')

Extraneous columns are:
[]
2771 ALFs, 2771 unique Facility IDs, 0 NA Facility IDs


In [348]:
df_mn['State Facility Type 1 Literal'].unique()

array(['SUPERVISED LIVING FACILITY', 'HOUSING WITH SERVICES'],
      dtype=object)

In [350]:
df_mn = df_mn[df_mn['State Facility Type 1 Literal'] == 'HOUSING WITH SERVICES']

In [351]:
df_mn.to_csv('alf-datasets/mn/mn-alf-final.csv', index=False)

## Michigan

In [337]:
! ls alf-datasets/mi

column-descriptions.png mi-alf-hc.csv
mi-alf-final.csv        mi-alf-raw.csv


In [338]:
df_mi = pd.read_csv('alf-datasets/mi/mi-alf-hc.csv')

df_mi['Date Accessed'] = '6/28/21'
df_mi['State Facility Type 1 Literal'] = 'Adult Foster Care & Homes for the Aged facility'

In [339]:
df_mi['State Facility Type 2 Literal'].unique()

array(['AS', 'AL', 'AM', 'AF', 'AH', 'XH', 'AG', 'AI'], dtype=object)

In [340]:
def type_simplifier_mi(x):
    if x == 'AF': return 'Family Home'
    elif x == 'AS': return 'Small Group'
    elif x == 'AM': return 'Medium Group'
    elif x == 'AL': return 'Large Group'
    elif x == 'AG': return 'Congregate'
    elif x == 'AI': return 'County Infirmary'
    elif x == 'AH': return 'Home for the Aged'
    else: return x + " (couldn't find documentation on what this is)"

In [341]:
df_mi['State Facility Type 2 Literal'] = [type_simplifier_mi(x) for x in df_mi['State Facility Type 2 Literal']]

In [342]:
df_mi['Address'] = ['' if type(x) == float else x+', ' for x in df_mi['Address 1']] + df_mi['Address 2']

In [343]:
df_mi = df_mi.drop(columns=['Address 1', 'Address 2'])

In [344]:
cleaner(df_mi, 'MI')

Extraneous columns are:
[]
4591 ALFs, 4591 unique License Numbers, 0 NA License Numbers


In [345]:
df_mi.to_csv('alf-datasets/mi/mi-alf-final.csv', index=False)

## Massachusetts

In [319]:
! ls alf-datasets/ma

ma-alf-final.csv ma-alf-hc.csv    ma-alf-raw.csv


In [320]:
df_ma = pd.read_csv('alf-datasets/ma/ma-alf-hc.csv')
df_ma['Date Accessed'] = '6/28/21'
df_ma['State Facility Type 1 Literal'] = 'assisted living residence'
df_ma['Capacity'] = df_ma['Capacity 1'] + df_ma['Capacity 2']
df_ma = df_ma.drop(columns=['Capacity 1', 'Capacity 2'])
cleaner(df_ma, 'MA')

Extraneous columns are:
[]
No License Number of Facility ID


In [321]:
df_ma.to_csv('alf-datasets/ma/ma-alf-final.csv', index=False)

## Maryland 

In [1013]:
! ls alf-datasets/md

md-alf-final.csv md-alf-hc.csv    md-alf-raw.csv


In [1014]:
df_md = pd.read_csv('alf-datasets/md/md-alf-hc.csv')
df_md['Date Accessed'] = '6/28/21'
cleaner(df_md, 'MD')

Extraneous columns are:
[]
No License Number of Facility ID


In [1015]:
df_md.to_csv('alf-datasets/md/md-alf-final.csv', index=False)

## Maine

In [1016]:
! ls alf-datasets/me

me-alf-final.csv me-alf-hc.csv    me-alf-raw.csv


In [1017]:
df_me = pd.read_csv('alf-datasets/me/me-alf-hc.csv')
df_me['Date Accessed'] = '7/12/21'
df_me = df_me[df_me['Status'] != 'FAILED TO RENEW']
df_me = df_me.drop('Status', axis=1)
cleaner(df_me, 'ME')

Extraneous columns are:
[]
1086 ALFs, 1086 unique License Numbers, 0 NA License Numbers


In [1018]:
df_me.to_csv('alf-datasets/me/me-alf-final.csv', index=False)

## Louisiana

In [1019]:
! ls alf-datasets/la

la-alf-final.csv la-alf-hc.csv    la-alf-raw.csv


In [1020]:
df_la = pd.read_csv('alf-datasets/la/la-alf-hc.csv')
df_la = df_la[df_la['State Facility Type 1 Literal'] == 'Adult Residential Care']
df_la['Date Accessed'] = '6/25/21'
cleaner(df_la, 'LA')

Extraneous columns are:
[]
159 ALFs, 159 unique License Numbers, 0 NA License Numbers


In [1021]:
df_la.to_csv('alf-datasets/la/la-alf-final.csv', index=False)

## Kentucky

In [37]:
! ls alf-datasets/ky

ky-alf-al-hc.csv               ky-alf-freestanding-pc-hc.csv
ky-alf-al-raw.csv              ky-alf-freestanding-pc-raw.csv
ky-alf-final.csv


In [38]:
df_ky_pch = pd.read_csv('alf-datasets/ky/ky-alf-freestanding-pc-hc.csv')
df_ky_al = pd.read_csv('alf-datasets/ky/ky-alf-al-hc.csv')

df_ky_pch['State Facility Type 1 Literal'] = 'Personal Care Home'
df_ky_pch['State Facility Type 2 Literal'] = 'Freestanding'
df_ky_al['State Facility Type 1 Literal'] = 'Assisted Living Community'

df_ky = pd.concat([df_ky_pch, df_ky_al], ignore_index=True)
df_ky['Date Accessed'] = '6/25/21'

cleaner(df_ky, 'KY')

Extraneous columns are:
[]
216 ALFs, 106 unique License Numbers, 111 NA License Numbers


In [41]:
df_ky[df_ky['State Facility Type 1 Literal'] == 'ASSISTED LIVING COMMUNITY'].to_csv('alf-datasets/ky/ky-alf-final.csv', index=False)

In [42]:
df_ky.to_csv('alf-datasets/ky/ky-alf-final-with-extra-facilities.csv', index=False)

## Kansas

In [12]:
! ls alf-datasets/ks

ks-alf-al-hc.csv    ks-alf-final.csv    ks-alf-hp-raw.csv   ks-alf-rhcf-raw.csv
ks-alf-al-raw.csv   ks-alf-hp-hc.csv    ks-alf-rhcf-hc.csv


In [13]:
df_ks_al = pd.read_csv('alf-datasets/ks/ks-alf-al-hc.csv')
df_ks_rhcf = pd.read_csv('alf-datasets/ks/ks-alf-rhcf-hc.csv')
df_ks_hp = pd.read_csv('alf-datasets/ks/ks-alf-hp-hc.csv')

df_ks = pd.concat([df_ks_al, df_ks_rhcf, df_ks_hp], ignore_index=True)
df_ks['Date Accessed'] = '6/25/21'
# df_ks = df_ks.rename(columns={'Zipcode':'Zip Code'})

In [14]:
df_ks['Facility Name'] = df_ks['Facility Name 2'].combine(df_ks['Licensee'], lambda x, y: y if x == '-' else x).str.title()

In [15]:
df_ks = df_ks.drop(columns='Facility Name 2')

In [16]:
cleaner(df_ks, 'KS')

Extraneous columns are:
['Zipcode']
474 ALFs, 474 unique Facility IDs, 0 NA Facility IDs


In [18]:
df_ks.drop(columns='Zip Code', inplace=True)
df_ks.rename(columns={'Zipcode':'Zip Code'}, inplace=True)

In [20]:
df_ks.to_csv('alf-datasets/ks/ks-alf-final.csv', index=False)

## Iowa

In [28]:
! ls alf-datasets/ia

ia-alf-final.csv ia-alf-hc.csv    ia-alf-raw.csv


In [29]:
df_ia = pd.read_csv('alf-datasets/ia/ia-alf-hc.csv')
df_ia = df_ia[df_ia['Status'] == 'Active']
df_ia = df_ia.drop(columns='Status')
df_ia['Date Accessed'] = '6/24/21'
cleaner(df_ia, 'IA')

Extraneous columns are:
[]
339 ALFs, 339 unique Facility IDs, 0 NA Facility IDs


In [35]:
df_ia[df_ia['State Facility Type 1 Literal'] == 'ASSISTED LIVING PROGRAMS'].to_csv('alf-datasets/ia/ia-alf-final.csv', index=False)

In [36]:
df_ia.to_csv('alf-datasets/ia/ia-alf-final-with-extra-facilities.csv', index=False)

## Indiana

In [1054]:
! ls alf-datasets/in

in-alf-final.csv in-alf-hc.csv    in-alf-raw.csv


In [1055]:
df_in = pd.read_csv('alf-datasets/in/in-alf-hc.csv')
df_in['State Facility Type 2 Literal'] = df_in['Type 3'].str.cat(df_in['Type 2'], sep=', ')
df_in = df_in.drop(columns=['Type 2', 'Type 3'])
df_in['Date Accessed'] = '7/7/21'
cleaner(df_in, 'IN')

Extraneous columns are:
[]
364 ALFs, 364 unique Facility IDs, 0 NA Facility IDs


In [1057]:
df_in.to_csv('alf-datasets/in/in-alf-final.csv', index=False)

## Illinois

In [1058]:
! ls alf-datasets/il

il-alf-final.csv il-alf-hc.csv    il-alf-raw.csv


In [1059]:
df_il = pd.read_csv('alf-datasets/il/il-alf-hc.csv')
df_il['Date Accessed'] = '6/24/21'
df_il['State Facility Type 1 Literal'] = 'Assisted Living/Shared Housing'
cleaner(df_il, 'IL')

Extraneous columns are:
[]
495 ALFs, 495 unique License Numbers, 0 NA License Numbers


In [1060]:
df_il.to_csv('alf-datasets/il/il-alf-final.csv', index=False)

## Hawaii

In [21]:
! ls alf-datasets/hi

hi-alf-al-hc.csv    hi-alf-al.pdf       hi-alf-arch-raw.csv hi-alf-final.csv
hi-alf-al-raw.csv   hi-alf-arch-hc.csv  hi-alf-arch.pdf


In [22]:
df_hi_al = pd.read_csv('alf-datasets/hi/hi-alf-al-hc.csv')
df_hi_arch = pd.read_csv('alf-datasets/hi/hi-alf-arch-hc.csv')

df_hi_al['State Facility Type 1 Literal'] = 'Assisted Living Facility'

df_hi = pd.concat([df_hi_al, df_hi_arch], ignore_index=True)

df_hi['Date Accessed'] = '6/24/21'
cleaner(df_hi, 'HI')

Extraneous columns are:
['Island']
474 ALFs, 474 unique License Numbers, 0 NA License Numbers


In [23]:
df_hi = df_hi.drop(columns='Island')

In [26]:
# just ALF to follow NCAL study
df_hi[df_hi['State Facility Type 1 Literal'] == 'ASSISTED LIVING FACILITY'].to_csv('alf-datasets/hi/hi-alf-final.csv', index=False)

In [27]:
# df with extras
df_hi.to_csv('alf-datasets/hi/hi-alf-final-with-extra-facilities.csv', index=False)

## Georgia

In [1064]:
! ls alf-datasets/ga

ga-alf-final.csv ga-alf-hc.csv    ga-alf-raw.csv


In [1065]:
df_ga = pd.read_csv('alf-datasets/ga/ga-alf-hc.csv')
df_ga['Date Accessed'] = '6/24/21'
cleaner(df_ga, 'GA')

Extraneous columns are:
[]
No License Number of Facility ID


In [1066]:
df_ga.to_csv('alf-datasets/ga/ga-alf-final.csv', index=False)

## Florida

In [1067]:
! ls alf-datasets/fl

fl-alf-final.csv fl-alf-hc.csv    fl-alf-raw.csv


In [1068]:
df_fl = pd.read_csv('alf-datasets/fl/fl-alf-hc.csv')
df_fl['Date Accessed'] = '6/24/21'
cleaner(df_fl, 'FL')

Extraneous columns are:
[]
3157 ALFs, 3155 unique License Numbers, 1 NA License Numbers


In [1069]:
df_fl.to_csv('alf-datasets/fl/fl-alf-final.csv', index=False)

## Connecticut

In [1070]:
! ls alf-datasets/ct

ct-alf-al-hc.csv   ct-alf-final.csv   ct-alf-rcf-raw.csv
ct-alf-al-raw.csv  ct-alf-rcf-hc.csv


In [1071]:
df_ct_al = pd.read_csv('alf-datasets/ct/ct-alf-al-hc.csv')
df_ct_rcf = pd.read_csv('alf-datasets/ct/ct-alf-rcf-hc.csv')

df_ct_al['State Facility Type 1 Literal'] = 'Assisted Living Service Agency'
df_ct_rcf['State Facility Type 1 Literal'] = 'Residential Care Facility'

df_ct = pd.concat([df_ct_al, df_ct_rcf], ignore_index=True)

df_ct['Date Accessed'] = '7/13/21'
cleaner(df_ct, 'CT')

Extraneous columns are:
[]
210 ALFs, 210 unique License Numbers, 0 NA License Numbers


In [1072]:
df_ct.to_csv('alf-datasets/ct/ct-alf-final.csv', index=False)

## Colorado

In [1073]:
! ls alf-datasets/co

co-alf-final.csv co-alf-hc.csv    co-alf-raw.csv


In [1074]:
df_co = pd.read_csv('alf-datasets/co/co-alf-hc.csv')
df_co['Date Accessed'] = '7/6/21'
cleaner(df_co, 'CO')

Extraneous columns are:
[]
711 ALFs, 711 unique Facility IDs, 0 NA Facility IDs


In [1075]:
df_co.to_csv('alf-datasets/co/co-alf-final.csv', index=False)

## California

In [1076]:
! ls alf-datasets/ca

ca-alf-final.csv ca-alf-hc.csv    ca-alf-raw.csv


In [1077]:
df_ca = pd.read_csv('alf-datasets/ca/ca-alf-hc.csv')
df_ca = df_ca[df_ca['Facility Status'] != 'CLOSED']
df_ca = df_ca.drop(columns='Facility Status')
df_ca['Date Accessed'] = '7/13/21'
cleaner(df_ca, 'CA')

Extraneous columns are:
[]
8042 ALFs, 8042 unique Facility IDs, 0 NA Facility IDs


In [1078]:
df_ca.to_csv('alf-datasets/ca/ca-alf-final.csv', index=False)

## Arizona

In [1079]:
! ls alf-datasets/az

az-alf-final.csv az-alf-hc.csv    az-alf-raw.csv


In [1080]:
df_az = pd.read_csv('alf-datasets/az/az-alf-hc.csv')
df_az['Date Accessed'] = '7/13/21'
df_az = df_az[df_az['State Facility Type 1 Literal'].apply(lambda x: 'ASSISTED LIVING' in x)]
cleaner(df_az, 'AZ')

Extraneous columns are:
[]
2113 ALFs, 2113 unique License Numbers, 0 NA License Numbers


In [1081]:
df_az.to_csv('alf-datasets/az/az-alf-final.csv', index=False)

## Alabama

In [1082]:
! ls alf-datasets/al

al-alf-al-hc.csv  al-alf-final.csv  al-alf-sc-raw.csv
al-alf-al-raw.csv al-alf-sc-hc.csv  al-pums.json


In [1083]:
df_al_al = pd.read_csv('alf-datasets/al/al-alf-al-hc.csv')
df_al_sc = pd.read_csv('alf-datasets/al/al-alf-sc-hc.csv')
df_al = pd.concat([df_al_al,df_al_sc], ignore_index=True)
df_al['Date Accessed'] = '6/24/21'

In [1084]:
df_al['Address'] = df_al['Address 1'] + ['' if type(x) == float else ', '+x for x in df_al['Address 2']]
df_al = df_al.drop(columns=['Address 1', 'Address 2'])

In [1085]:
cleaner(df_al, 'AL')

Extraneous columns are:
[]
300 ALFs, 300 unique Facility IDs, 0 NA Facility IDs


In [1086]:
df_al.to_csv('alf-datasets/al/al-alf-final.csv', index=False)

## Vermont

In [1087]:
df_vt_al = pd.read_csv('alf-datasets/vt/vt-alf-al-hc.csv')
df_vt_rch = pd.read_csv('alf-datasets/vt/vt-alf-rch-hc.csv')

df_vt_al['State Facility Type 1 Literal'] = 'Assisted Living Residence'
df_vt_rch['State Facility Type 1 Literal'] = 'Residential Care Home'

df_vt = pd.concat([df_vt_al, df_vt_rch], ignore_index=True)
cleaner(df_vt, 'VT')

Extraneous columns are:
[]
No License Number of Facility ID


In [1088]:
df_vt.to_csv('alf-datasets/vt/vt-alf-final.csv', index=False)

## Alaska

In [4]:
!ls alf-datasets/ak

ak-alf-hc.csv  ak-alf-raw.csv


In [6]:
df_ak = pd.read_csv('alf-datasets/ak/ak-alf-hc.csv')
df_ak['Date Accessed'] = '8/2/21'
df_ak['State Facility Type 1 Literal'] = 'Assisted living home'

In [7]:
cleaner(df_ak, 'AK')

Extraneous columns are:
[]
704 ALFs, 704 unique License Numbers, 0 NA License Numbers


In [10]:
df_ak.to_csv('alf-datasets/ak/ak-alf-final.csv', index=False)

## Montana

In [14]:
! ls alf-datasets/mt/

mt-alf-hc.csv  mt-alf-raw.csv


In [15]:
df_mt = pd.read_csv('alf-datasets/mt/mt-alf-hc.csv')
df_mt['Date Accessed'] = '8/2/21'

cleaner(df_mt, 'MT')

Extraneous columns are:
['City\n', 'Zip Code\n', 'State\xa0Facility\xa0Type\xa01\xa0Literal', 'State\n']
213 ALFs, 213 unique License Numbers, 0 NA License Numbers


In [16]:
df_mt.columns

Index(['State Facility Type 1 Literal', 'License Number', 'Facility Name',
       'Phone Number', 'Capacity', 'County', 'Address', 'City\n', 'State\n',
       'Zip Code\n', 'Date Accessed', 'State'],
      dtype='object')

In [17]:
df_mt.columns = ['State Facility Type 1 Literal', 'License Number', 'Facility Name', 'Phone Number', 'Capacity', 'County', 'Address', 'City', 'State', 'Zip Code', 'Date Accessed', 'State']

In [18]:
cleaner(df_mt, 'MT')

Extraneous columns are:
[]
213 ALFs, 213 unique License Numbers, 0 NA License Numbers


In [20]:
df_mt.to_csv('alf-datasets/mt/mt-alf-final.csv', index=False)

## New Hampshire

In [21]:
! ls alf-datasets/nh

nh-alf-hc.csv  nh-alf-raw.csv nh-alf.pdf


In [43]:
df_nh = pd.read_csv('alf-datasets/nh/nh-alf-hc.csv')
df_nh['Date Accessed'] = '8/3/21'
df_nh['State Facility Type 1 Literal'] = 'assisted living residence'

df_nh = df_nh[df_nh['TYPE'].isin(['Supported Residential Care Facility', 'Assist Living/Residential Care Facility'])]
df_nh['State Facility Type 2 Literal'] = [x.split('/')[1] if x == 'Assist Living/Residential Care Facility' else x for x in df_nh['TYPE']]
df_nh.drop(columns='TYPE', inplace=True)

cleaner(df_nh, 'NH')

Extraneous columns are:
[]
138 ALFs, 138 unique License Numbers, 0 NA License Numbers


In [45]:
df_nh.to_csv('alf-datasets/nh/nh-alf-final.csv', index=False)

# Extracting from HTML
Code to extra data from states where data is only accessible in an HTML document

## Delaware

In [208]:
! ls alf-datasets/de

de-alf-final.csv de-alf.html


In [209]:
soup = BeautifulSoup(open('alf-datasets/de/de-alf.html', 'r').read(), 'html.parser')

In [210]:
# finding the first ALF to see what HTML looks like
# soup.prettify().find('Arden Courts of Wilmington')
# checking out what the HTML looks like to see what we should search for
# print(soup.prettify()[21400:22500])

In [211]:
soup_list = soup.find(id='main_content').find('ul').find_all('li')

# gets string of each <li> element, unicode normalizes it, then splits based on '\n\n\n\n'
df_de = pd.Series([unicodedata.normalize('NFKD', stuff.text) for stuff in soup_list]).str.split('\n\n\n\n', expand=True)

In [212]:
df_de.head()

,0,1,2,3,4
0,Arden Courts of Wilmington,700 1/2 Foulk Rd.,"Wilmington, DE 19803-3708",302-762-7800,State Licensed Beds: 56\n
1,Brandywine Senior Living at Fenwick Island,21111 Arrington Drive,"Selbyville, DE 19975",302-436-0808,State Licensed Beds: 125\n
2,Brandywine Senior Living at Seaside Pointe,36101 Seaside Blvd.,"Rehoboth Beach, DE 19971-1189",302-226-8750,State Licensed Beds: 150\n
3,Brookdale Dover,150 Saulsbury Road,"Dover, DE 19904-2776",302-674-4407,State Licensed Beds: 96\n
4,Brookdale Hockessin,6677 Lancaster Pike,"Hockessin, DE 19707-9503",302-239-3200,State Licensed Beds: 66\n


In [213]:
df_de['City'] = df_de[2].str.split(',', expand=True)[0]
df_de['Zip Code'] = df_de[2].str.split(',', expand=True)[1].str.split(' ', expand=True)[3]
df_de['Capacity'] = df_de[4].str.split(': ', expand=True)[1].str.replace('\n', '').astype(int)
df_de.columns = ['Facility Name', 'Address', 'REMOVE', 'Phone Number', 'REMOVE', 'City', 'Zip Code', 'Capacity']

In [214]:
# checking columns were made correctly
df_de.head()

,Facility Name,Address,REMOVE,Phone Number,REMOVE,City,Zip Code,Capacity
0,Arden Courts of Wilmington,700 1/2 Foulk Rd.,"Wilmington, DE 19803-3708",302-762-7800,State Licensed Beds: 56\n,Wilmington,19803-3708,56
1,Brandywine Senior Living at Fenwick Island,21111 Arrington Drive,"Selbyville, DE 19975",302-436-0808,State Licensed Beds: 125\n,Selbyville,19975,125
2,Brandywine Senior Living at Seaside Pointe,36101 Seaside Blvd.,"Rehoboth Beach, DE 19971-1189",302-226-8750,State Licensed Beds: 150\n,Rehoboth Beach,19971-1189,150
3,Brookdale Dover,150 Saulsbury Road,"Dover, DE 19904-2776",302-674-4407,State Licensed Beds: 96\n,Dover,19904-2776,96
4,Brookdale Hockessin,6677 Lancaster Pike,"Hockessin, DE 19707-9503",302-239-3200,State Licensed Beds: 66\n,Hockessin,19707-9503,66


In [215]:
df_de.drop(columns='REMOVE', inplace=True)
df_de['Date Accessed'] = '6/24/21'
df_de['State Facility Type 1 Literal'] = 'assisted living facility'
cleaner(df_de, 'DE')

Extraneous columns are:
[]
No License Number of Facility ID


In [216]:
df_de.head()

,Facility Name,Address,Phone Number,City,Zip Code,Capacity,Date Accessed,State Facility Type 1 Literal,State
0,Arden Courts Of Wilmington,700 1/2 FOULK RD.,302-762-7800,Wilmington,19803-3708,56,6/24/21,ASSISTED LIVING FACILITY,DE
1,Brandywine Senior Living At Fenwick Island,21111 ARRINGTON DRIVE,302-436-0808,Selbyville,19975,125,6/24/21,ASSISTED LIVING FACILITY,DE
2,Brandywine Senior Living At Seaside Pointe,36101 SEASIDE BLVD.,302-226-8750,Rehoboth Beach,19971-1189,150,6/24/21,ASSISTED LIVING FACILITY,DE
3,Brookdale Dover,150 SAULSBURY ROAD,302-674-4407,Dover,19904-2776,96,6/24/21,ASSISTED LIVING FACILITY,DE
4,Brookdale Hockessin,6677 LANCASTER PIKE,302-239-3200,Hockessin,19707-9503,66,6/24/21,ASSISTED LIVING FACILITY,DE


In [217]:
df_de.to_csv('alf-datasets/de/de-alf-final.csv', index=False)

## New Mexico

In [280]:
! ls alf-datasets/nm

nm-alf.html


In [281]:
soup = BeautifulSoup(open('alf-datasets/nm/nm-alf.html', 'r').read(), 'html.parser')

In [282]:
# finding the first ALF to see what HTML looks like
# soup.prettify().find('The Village At Alameda')

In [283]:
# checking out what the HTML looks like to see what we should search for
# print(soup.prettify()[31000:33000])

In [284]:
# this is what we're looking for: <table bgcolor="#EEEEEE" width="586">
soup_list = soup.find('table', {'bgcolor':'#EEEEEE', 'width':'586'}).find_all('td', {'class':'H12_violet_leftn'})
soup_list = [unicodedata.normalize('NFKD', stuff.text) for stuff in soup_list]

df_nm = pd.DataFrame(np.reshape(soup_list, (-1, 11)))

In [286]:
df_nm.head()

,0,1,2,3,4,5,6,7,8,9,10
0,The Village At Alameda,8810 HORIZON BOULEVARD NE,ALBUQUERQUE NM 87113,(505) 857-7000 Fax: (505) 857-9210,,,2244,12/31/2021,MONICA COLLINS,ALBUQUERQUE OPERATING COMPANY LLC,86
1,"A LOVE FOR LIFE, LLC",7534 BEAR CANYON ROAD NE,ALBUQUERQUE NM 87109,(505) 350-2563 Fax: (505) 884-1848,,,2188,07/31/2021,ROANN SEXSON,"A LOVE FOR LIFE, LLC",10
2,"A Love For Life, Llc",4108 ALCAZAR NE,ALBUQUERQUE NM 87109,(505) 884-3912 Fax: (505) 884-1848,,,2278,05/31/2022,ROANN SEXSON,"A LOVE FOR LIFE, LLC",15
3,A NEW DAY,11212 MIRAVISTA PLACE SE,ALBUQUERQUE NM 87123,(505) 263-0480 Fax: ( ) -,,,2126,08/31/2021,ATENOGENES VAZQUEZ,ATENOGENES VAZQUEZ,3
4,A New Day Assisted Living #2,11208 MIRA VISTA PLACE SE,ALBUQUERQUE NM 87123,(505) 373-9285 Fax: ( ) -,,,2277,05/31/2022,ATENOGENES VAZQUEZ,ATENOGENES VAZQUEZ,3


In [311]:
df_nm['City'] = df_nm[2].str.split(' ', expand=True)[0]
df_nm['Zip Code'] = df_nm[2].str.split(' ', expand=True)[4]
df_nm['Phone Number'] = df_nm[3].str.split('Fax:', expand=True)[0]
df_nm['Capacity'] = pd.to_numeric(df_nm[10], errors='coerce')

In [312]:
df_nm.head()

,0,1,2,3,4,5,6,7,8,9,10,City,Zip Code,Phone Number,Capacity
0,The Village At Alameda,8810 HORIZON BOULEVARD NE,ALBUQUERQUE NM 87113,(505) 857-7000 Fax: (505) 857-9210,,,2244,12/31/2021,MONICA COLLINS,ALBUQUERQUE OPERATING COMPANY LLC,86,ALBUQUERQUE,87113,(505) 857-7000,86.0
1,"A LOVE FOR LIFE, LLC",7534 BEAR CANYON ROAD NE,ALBUQUERQUE NM 87109,(505) 350-2563 Fax: (505) 884-1848,,,2188,07/31/2021,ROANN SEXSON,"A LOVE FOR LIFE, LLC",10,ALBUQUERQUE,87109,(505) 350-2563,10.0
2,"A Love For Life, Llc",4108 ALCAZAR NE,ALBUQUERQUE NM 87109,(505) 884-3912 Fax: (505) 884-1848,,,2278,05/31/2022,ROANN SEXSON,"A LOVE FOR LIFE, LLC",15,ALBUQUERQUE,87109,(505) 884-3912,15.0
3,A NEW DAY,11212 MIRAVISTA PLACE SE,ALBUQUERQUE NM 87123,(505) 263-0480 Fax: ( ) -,,,2126,08/31/2021,ATENOGENES VAZQUEZ,ATENOGENES VAZQUEZ,3,ALBUQUERQUE,87123,(505) 263-0480,3.0
4,A New Day Assisted Living #2,11208 MIRA VISTA PLACE SE,ALBUQUERQUE NM 87123,(505) 373-9285 Fax: ( ) -,,,2277,05/31/2022,ATENOGENES VAZQUEZ,ATENOGENES VAZQUEZ,3,ALBUQUERQUE,87123,(505) 373-9285,3.0


In [313]:
df_nm.columns = ['Facility Name', 'Address', 'REMOVE', 'REMOVE', 'REMOVE', 'REMOVE', 'License Number', 'REMOVE', 'REMOVE', 'Licensee', 'REMOVE', 'City', 'Zip Code', 'Phone Number', 'Capacity' ]

In [315]:
df_nm.drop(columns='REMOVE', inplace=True)
df_nm.head()

,Facility Name,Address,License Number,Licensee,City,Zip Code,Phone Number,Capacity
0,The Village At Alameda,8810 HORIZON BOULEVARD NE,2244,ALBUQUERQUE OPERATING COMPANY LLC,ALBUQUERQUE,87113,(505) 857-7000,86.0
1,"A LOVE FOR LIFE, LLC",7534 BEAR CANYON ROAD NE,2188,"A LOVE FOR LIFE, LLC",ALBUQUERQUE,87109,(505) 350-2563,10.0
2,"A Love For Life, Llc",4108 ALCAZAR NE,2278,"A LOVE FOR LIFE, LLC",ALBUQUERQUE,87109,(505) 884-3912,15.0
3,A NEW DAY,11212 MIRAVISTA PLACE SE,2126,ATENOGENES VAZQUEZ,ALBUQUERQUE,87123,(505) 263-0480,3.0
4,A New Day Assisted Living #2,11208 MIRA VISTA PLACE SE,2277,ATENOGENES VAZQUEZ,ALBUQUERQUE,87123,(505) 373-9285,3.0


In [316]:
df_nm['Date Accessed'] = '6/29/21'
cleaner(df_nm, 'NM')

Extraneous columns are:
[]
250 ALFs, 250 unique License Numbers, 0 NA License Numbers


In [317]:
df_nm.head()

,Facility Name,Address,License Number,Licensee,City,Zip Code,Phone Number,Capacity,Date Accessed,State
0,The Village At Alameda,8810 HORIZON BOULEVARD NE,2244,ALBUQUERQUE OPERATING COMPANY LLC,Albuquerque,87113,(505) 857-7000,86.0,6/29/21,NM
1,"A Love For Life, Llc",7534 BEAR CANYON ROAD NE,2188,"A LOVE FOR LIFE, LLC",Albuquerque,87109,(505) 350-2563,10.0,6/29/21,NM
2,"A Love For Life, Llc",4108 ALCAZAR NE,2278,"A LOVE FOR LIFE, LLC",Albuquerque,87109,(505) 884-3912,15.0,6/29/21,NM
3,A New Day,11212 MIRAVISTA PLACE SE,2126,ATENOGENES VAZQUEZ,Albuquerque,87123,(505) 263-0480,3.0,6/29/21,NM
4,A New Day Assisted Living #2,11208 MIRA VISTA PLACE SE,2277,ATENOGENES VAZQUEZ,Albuquerque,87123,(505) 373-9285,3.0,6/29/21,NM


In [318]:
df_nm.to_csv('alf-datasets/nm/nm-alf-final.csv', index=False)

## Idaho

In [603]:
! ls alf-datasets/id

id-alf.html


In [604]:
soup = BeautifulSoup(open('alf-datasets/id/id-alf.html', 'r').read(), 'html.parser')

In [605]:
# finding the first ALF to see what HTML looks like
# soup.prettify().find('Aarenbrooke Place - Cory Lane')

In [606]:
# checking out what the HTML looks like to see what we should search for
# print(soup.prettify()[149000:154000])

In [607]:
# want to find item with: <table border="1" cellspacing="0" class="dg" id="dg" rules="all" style="width:1250px;border-collapse:collapse;">
soup_list = soup.find('table', {'border':'1', 'cellspacing':'0', 'style':'width:1250px;border-collapse:collapse;'}).find_all('td')[10:]
soup_list = [unicodedata.normalize('NFKD', stuff.text) for stuff in soup_list]

In [608]:
# not divisible by 9 so we add one more item to end
len(soup_list)

2447

In [609]:
soup_list.append('filler')
df_id = pd.DataFrame(np.reshape(soup_list, (-1, 9)))

In [610]:
df_id.drop(columns=[5,6,7,8], inplace=True)
df_id.columns = ['Facility Name', 'Address', 'City', 'Phone Number', 'Capacity']

In [613]:
df_id['Date Accessed'] = '6/24/21'
df_id['State Facility Type 1 Literal'] = 'residentialcare/assisted living facility'
cleaner(df_id, 'ID')

Extraneous columns are:
[]
No License Number of Facility ID


In [615]:
df_id.to_csv('alf-datasets/id/id-alf-final.csv', index=False)

## North Carolina

In [944]:
! ls alf-datasets/nc

nc-alf-ach-hc.csv  nc-alf-ach-raw.csv nc-alf-final.csv   nc-alf-mahs.html


In [945]:
soup = BeautifulSoup(open('alf-datasets/nc/nc-alf-mahs.html', 'r').read(), 'html.parser')

In [946]:
# finding the first ALF to see what HTML looks like
soup.prettify().find('Zenith Manor')

8173

In [947]:
# checking out what the HTML looks like to see what we should search for
# print(soup.prettify()[6000:10000])

In [948]:
# want to find all <tbody> in the HTML file, and then all <tr> in each <tbody>
# probably better way to do this
soup_list = soup.find_all('tbody')
soup_list2 = [a.find_all('tr') for a in soup_list]
soup_list3 = [item for sublist in soup_list2 for item in sublist]
soup_list4 = [a.find_all('td') for a in soup_list3]

# split each string with any number of \n's
# df_nc_mahs = pd.Series([unicodedata.normalize('NFKD', stuff.text) for stuff in soup_list_long_flattened]).str.split(r"(\n)+", expand=True).iloc[:, [2,4,6,8,10,12,14,16,18,20]]
df_nc_mahs = pd.DataFrame(soup_list4).applymap(lambda x: unicodedata.normalize('NFKD', x.text).strip('\n') if x else x)

In [949]:
# adding up 1, 2, and more than 2 bedrooms as contributing 1,2, and 3 capacity
df_nc_mahs['Capacity'] = pd.to_numeric(df_nc_mahs[6], errors='coerce').fillna(0) + 2*pd.to_numeric(df_nc_mahs[7], errors='coerce').fillna(0) + 3*pd.to_numeric(df_nc_mahs[8], errors='coerce').fillna(0)

df_nc_mahs['City'] = df_nc_mahs[2].str.split(',', expand=True)[0]
df_nc_mahs['Zip Code'] = df_nc_mahs[2].str[-5:]
df_nc_mahs['State'] = 'NC'
df_nc_mahs['Date Accessed'] = '8/17/21'
df_nc_mahs['State Facility Type 1 Literal'] = 'ASSISTED LIVING RESIDENCE'
df_nc_mahs['State Facility Type 2 Literal'] = 'Multi-unit Assisted Housing with Services Facilities'

In [950]:
df_nc_mahs.drop(columns=[2,4,5,6,7,8,9], inplace=True)
df_nc_mahs.rename(columns={0:'Facility Name', 1:'Address', 3:'Phone Number'}, inplace=True)

In [951]:
df_nc = pd.concat([df_nc_mahs, df_nc_ach], ignore_index=True)
cleaner(df_nc, state='NC')

Extraneous columns are:
[]
693 ALFs, 595 unique License Numbers, 99 NA License Numbers


In [953]:
df_nc.to_csv('alf-datasets/nc/nc-alf-final.csv', index=False)